In [ ]:
#| default_exp pipelines

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

# Imports and setup

### Unconditional imports

In [ ]:
#| export

# print separately that we're loading dependencies, as this can take a while
# and we want to give immediate feedback the program is starting
from clip_plot.utils import timestamp

print(timestamp(), "Beginning to load dependencies")

2025-11-18 22:01:36.827642: Beginning to load dependencies


In [ ]:
#|export
from pathlib import Path
from shutil import rmtree

import numpy as np
import polars as pl
from fastcore.all import in_ipython

from clip_plot.configuration import Cfg, ClusterSpec, ImageLoaderOptions, UmapSpec, ViewerOptions
from clip_plot.embeddings import get_embeddings
from clip_plot.from_tables import cat_tables, table_to_meta
from clip_plot.images import ImageFactory, write_viewer_images
from clip_plot.metadata import get_manifest, write_metadata
from clip_plot.web_config import copy_web_assets, get_clip_plot_root



## Entry

`project_images` will kick off all the main functions for the module

In [ ]:
#| export

def project_images_pipeline(output_dir: Path,
                            plot_id: str,
                            model: str,
                            viewer_opts: ViewerOptions,
                            umap_spec: UmapSpec,
                            cluster_spec: ClusterSpec,
                            image_opts: ImageLoaderOptions,
                            images: list[Path] | None = None,
                            tables: list[Path] | None = None,
                            metadata: list[Path] | None = None,
                            image_path_col: str = "image_path",
                            vectors_col: str = "hidden_vectors",
                            x_col: str | None = None,
                            y_col: str | None = None,
        ):
        """Convert a folder of images into a clip-plot visualization"""

        # TODO: use a dataframe instead
        meta_names:list[str] = []
        meta_vals: list[dict] | None = None
        if tables and images:
                raise ValueError("Provide either tables or images parameter, not both.")
        if not tables and not images:
                raise ValueError("No images found from either tables or images input.")
        if tables and not images:
                print(timestamp(), "Loading tables")
                table: pl.DataFrame | None = cat_tables(tables)
                images: list[Path] = [Path(p) for p in table[image_path_col].to_numpy()]
                print(timestamp(), "Loading embeddings from disk")
                hidden_vectors: np.ndarray | None = table[vectors_col].to_numpy()
                meta_names, meta_vals= table_to_meta(table)
        elif not tables and images:
                hidden_vectors = get_embeddings(images, model_name=model)

        data_dir = output_dir / "data"
        imageEngine = ImageFactory(images, data_dir, metadata,
                                            **image_opts.model_dump())

        # TODO: simplify the mad tables/metadata possibilities
        if meta_vals:
                imageEngine.meta_headers = meta_names
                imageEngine.metadata = meta_vals

        print(f"Config to project images: {str(image_opts.model_dump())}")

        np.random.seed(image_opts.seed)
        print(timestamp(), "Starting image processing pipeline.")

        copy_web_assets(output_dir=output_dir,
                        tagline=viewer_opts.tagline, logo=viewer_opts.logo)

        atlas_positions = write_viewer_images(imageEngine, plot_id,
                        data_dir=data_dir, thumb_size=image_opts.thumbnail_size,
                        row_height=image_opts.atlas_row_height, atlas_size=image_opts.atlas_size)
        write_metadata(imageEngine)

        get_manifest(imageEngine, atlas_positions, hidden_vectors,
                     plot_id=plot_id, output_dir=output_dir,
                     umap_spec=umap_spec, cluster_spec=cluster_spec,
                     x_col=x_col, y_col=y_col,
        )
        # write_images(imageEngine)
        print(timestamp(), "Done!")

### Embed images without building visualization

Outputs a table linking the embedding files to the images

Test with bash command:
```
clipplot_embed_images --images "tests/smithsonian_butterflies_10/jpgs/*.jpg" --output_dir "tests/smithsonian_butterflies_10/DELETEME_embed_only" --metadata "tests/smithsonian_butterflies_10/meta_data/good_meta.csv"
```

In [ ]:
#| export

def embed_images_pipeline(images: list[Path],
                     model: str,
                     metadata: list[Path] | None,
                     output_dir: Path,
                     table_format: str,
                     table_id: str,
                ):
                """Embed a folder of images, save embeddings as .npy file to disk"""
                output_dir = Path(output_dir)
                data_dir = output_dir / "data"
                table_dir = output_dir / "data" / "tables"

                imageEngine = ImageFactory(image_paths=images, data_dir=data_dir, metadata_paths=metadata)

                embeddings = get_embeddings(image_paths=images, model_name=model)
                df = pl.DataFrame({"image_path": images,
                                   "image_filename": imageEngine.filenames,
                                   "hidden_vectors": embeddings,
                })

                if len(imageEngine.metadata) > 0:
                        df_meta = pl.DataFrame(imageEngine.metadata)
                        df_meta = df_meta.rename({"filename": "image_filename"})
                        # drop "image_path" column if df_meta has it
                        if "image_path" in df_meta.columns:
                                df_meta = df_meta.drop("image_path")

                        df = df.join(df_meta.unique(subset=["image_filename"]), on="image_filename",
                                              how="left")

                df = df.with_columns(pl.col("image_path").map_elements(
                                     lambda x: x.as_posix(),  return_dtype=pl.Utf8))

                ## standardize sort order of table
                # put standard columns in a sensible order if they exist in df
                standard_cols: set[str] = {"image_path", "image_filename", "hidden_vectors",
                                                   "category", "tags", "x", "y"}
                cols_in_order = list(standard_cols & set(df.columns))
                # append non-standard columns, sorted alphabetically
                non_standard_cols: list[str] = sorted(set(df.columns) - standard_cols)
                cols_sorted = cols_in_order + non_standard_cols
                df = df.with_columns(cols_sorted)

                table_dir.mkdir(parents=True, exist_ok=True)
                if table_format == "csv":
                        df.write_csv(table_dir / f"EmbedImages__{table_id}.csv")
                else:
                        df.write_parquet(table_dir / f"EmbedImages__{table_id}.parquet")

In [ ]:
#| hide

def test_butterfly():
    copy_root_dir = get_clip_plot_root()
    tests_dir = copy_root_dir.parents[0] / "tests"

    test_images = tests_dir / "smithsonian_butterflies_10/jpgs/*.jpg"
    test_output_dir = tests_dir / "smithsonian_butterflies_10/output_test_temp"
    meta = tests_dir / "smithsonian_butterflies_10/meta_data/good_meta.csv"
    if Path(test_output_dir).exists():
        rmtree(test_output_dir)

    cfg = Cfg(paths={"images": test_images},
                   model="../models/timm__vit_pe_core_tiny_patch16_384.fb",
                   cluster_spec={"min_cluster_size": 3},
                   )

    project_images_pipeline(images=cfg.paths.images,
                   output_dir=test_output_dir,
                   metadata=[meta],
                   plot_id="test_diff",
                   model=cfg.model,
                   viewer_opts = cfg.view_opts,
                   umap_spec = cfg.umap_spec,
                   cluster_spec = cfg.cluster_spec,
                   image_opts = cfg.image_opts,
                   )

    return

In [ ]:
#| hide

if __name__ == "__main__" and in_ipython():
    test_butterfly()

Device for inference: cpu


Device set to use cpu


2025-11-18 22:01:45.964702: Creating embeddings using ../models/timm__vit_pe_core_tiny_patch16_384.fb


  0%|          | 0/12 [00:00<?, ?it/s]

2025-11-18 22:01:47.758003: Done creating embeddings.
2025-11-18 22:01:47.761903: First three paths: /home/wsanger/git/clip-plot/tests/smithsonian_butterflies_10/jpgs/30aeb051d-ee0d-4c5b-8a85-a8da7baef5fd.jpg
/home/wsanger/git/clip-plot/tests/smithsonian_butterflies_10/jpgs/9fea3150-a3d4-11ed-aeea-e36f1256f233.jpg
/home/wsanger/git/clip-plot/tests/smithsonian_butterflies_10/jpgs/329a4c094-8536-4396-be70-3d9b5d0744d9.jpg
2025-11-18 22:01:47.762025: Validating input images


  0%|          | 0/12 [00:00<?, ?it/s]

AttributeError: 'ImageFactory' object has no attribute 'lod_cell_height'

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()